In [ ]:
#必要なライブラリのインポート
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
import math
import pathlib
import glob
import os
from pathlib import Path
from PIL import Image
#インライン表示
#%matplotlib inline

In [ ]:
#マスク画像を作成する関数
def create_mask(img):
    #画像の高さ、幅を取得
    height, width = img.shape[:2]
    
    #画像の囲まれたエリアを塗りつぶす
    for i in range(height):
        for j in range(width):
            if(img[i][j] == [47, 82, 143]).all():
                img[i][j] = [255, 255, 255]
            else:
                img[i][j] = [0, 0, 0]
                
    return img

In [ ]:
#回転した外接矩形の取得
def get_circumscribed_rectangle(mask_img):
    
    #2値化する
    ret, binary = cv2.threshold(mask_img, 127, 255, cv2.THRESH_BINARY)
    
    #膨張処理
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    binary = cv2.erode(binary, kernel)
    binary = cv2.dilate(binary, kernel)
    
    #輪郭抽出
    contours, hierarchy = cv2.findContours(
    binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    
    rects = []
    for cnt in contours:
        if cv2.contourArea(cnt) < 100:
            continue #小さい輪郭は無視する
            
        #回転した外接矩形を取る。
        rect = cv2.minAreaRect(cnt)
        rects.append(rect)
    
    return rects

In [ ]:
#計算した外接矩形の情報を用いて画像を回転させる。
def rotate_rectangle(img, rects):
    
    #外接矩形の情報を取得
    center, size, angle = rects
    print(angle)
    #float →　int
    center = tuple(map(int, center))
    size = tuple(map(int, size))
    
    #画像の高さ、幅
    height, width = img.shape[:2]
    
    #回転行列を作成
    M = cv2.getRotationMatrix2D(center, angle-90, 1)
    
    #画像を回転する
    rotated = cv2.warpAffine(img, M, (width, height))
    
    return rotated

In [ ]:
#回転したマスク画像の座標を計算してダーモス画像をクロップ
def crop_img(mask_img, dermo_img):
    
    #閾値127のバイナリ―imageを作成
    _,thresh = cv2.threshold(mask_img,127,255,cv2.THRESH_BINARY)
    
    #輪郭を見つける（白いboxを検出）
    contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    
    #長方形の輪郭の座標を抽出
    x,y,w,h = cv2.boundingRect(cnt)
    
    #計算した輪郭を用いて長方形にクロップして保存
    crop = dermo_img[y:y+h, x:x+w]

    return crop

In [ ]:
def save_rgb_data(img, dir_name):
    
    #保存先ディレクトリを作成
    output_dir = Path(dir_name +'/color_data')
    output_dir.mkdir(exist_ok = True)
    
    output_dir = str(output_dir)
    
    #3次元の色情報をCSVファイルに出力
    with open(output_dir + '/rgb.csv', 'wt') as f:
        writer = csv.writer(f)
        writer.writerows(img)
    
    #切り取った個所をRGB別に取得して保存
    img_b = img[:,:,0]
    img_g = img[:,:,1]
    img_r = img[:,:,2]
    
    with open(output_dir + '/r.csv', 'wt') as f:
        writer = csv.writer(f)
        writer.writerows(img_r)
        
    with open(output_dir + '/g.csv', 'wt') as f:
        writer = csv.writer(f)
        writer.writerows(img_g)
        
    with open(output_dir + '/b.csv', 'wt') as f:
        writer = csv.writer(f)
        writer.writerows(img_b)
    
    ### 色情報の列方向に対する平均化処理
    
    #RGB別に列方向対する平均値を取得
    ave_r = np.average(img_r, axis=0)
    ave_g = np.average(img_g, axis=0)
    ave_b = np.average(img_b, axis=0)
    
    #それぞれの色情報を整数値に丸める
    for i in range(ave_r.size):
        ave_r[i] = round(ave_r[i])
        ave_g[i] = round(ave_g[i])
        ave_b[i] = round(ave_b[i])
    
    #R,G,Bの色情報の平均を1つにまとめる(例 1000*3チャンネル)
    rgb_ave = []
    for i in range(ave_r.size):
        rgb_ave.append([ave_r[i], ave_g[i], ave_b[i]])
    
    #np.array型（int）に変換して保存
    rgb_ave = np.array(rgb_ave, dtype=np.int64)
    with open(output_dir + '/rgb_ave.csv', 'wt') as f:
        writer = csv.writer(f)
        writer.writerow(rgb_ave)

In [ ]:
def extract_color(file_path_dermo, file_path_box):
    
    #ダーモスコピー画像（青いboxあり）を読み込み
    dermo_box = cv2.imread(file_path_box)
    #画像をBGRからRGBへと変換
    dermo_box = cv2.cvtColor(dermo_box, cv2.COLOR_BGR2RGB)

    #マスク画像を作成して取得
    mask = create_mask(dermo_box)
        
    #画像のファイルのディレクトリを取得
    dir_name = os.path.dirname(file_path_dermo)
    
    #output用のフォルダを用意
    output_dir = Path(dir_name + '/outputs')
    output_dir.mkdir(exist_ok=True)
    output_dir = str(output_dir)
    
    cv2.imwrite(output_dir + '/mask.png', mask)
    
    #マスク画像をグレースケール化
    gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    
    #dermocopy読み込み(青いboxなし)
    dermo = cv2.imread(file_path_dermo)
    
    #マスク画像回転した外接矩形の情報を取得
    rects = get_circumscribed_rectangle(gray_mask)
    rect = rects[0]
    
    #外接矩形の情報を取得
    center, size, angle = rect
    
    if(angle != 0):
        #マスク画像の回転
        gray_mask = rotate_rectangle(gray_mask, rect)
        #ダーも画像の回転
        dermo = rotate_rectangle(dermo, rect)
        
        #回転後の画像を保存
        cv2.imwrite(output_dir + '/rotate_mask.png', gray_mask)
        cv2.imwrite(output_dir + '/rotate_dermo.png', dermo)
    
    
    #マスク画像の場所で切り抜き
    crop = crop_img(gray_mask, dermo)
    
    #画像の高さ、幅を取得
    height, width = crop.shape[:2]
    #画像の長方形が縦向きだった場合
    if(height > width):
        crop = cv2.rotate(crop, cv2.ROTATE_90_COUNTERCLOCKWISE)

    #切り抜いた画像の場所で保存
    cv2.imwrite(output_dir + "/crop_img.png", crop)
    
    #切り抜いた画像の保存
    save_rgb_data(crop, dir_name)

In [ ]:
# 絶対パスの取得
current_dir = pathlib.Path().resolve()
print(current_dir)

# ダーモの青いボックスとダーモ画像を取得
dermo_box = glob.glob(str(current_dir) + '/data/**/dermocopy_box*')
dermo = glob.glob(str(current_dir) + '/data/**/dermo[0-9]*')

test = str(current_dir) + '/data/21346/dermo21346.png'
test_box = str(current_dir) + '/data/21346/dermocopy_box21346.png'

extract_color(test, test_box)
#for i in range(len(dermo_box)):
#    extract_color(dermo[i], dermo_box[i])